In [11]:
import numpy as np
from numpy import array
# import matplotlib as plt
from random import sample
from numpy import zeros
from math import sqrt
import _pickle as cPickle
import h5py

In [12]:
# train = cPickle.load( open( "train_"+ str(0)+".pkl", "rb" ) )
# test = cPickle.load( open( "test_"+ str(0)+".pkl", "rb" ) )
# passive = cPickle.load( open( "passive_"+ str(51)+".pkl", "rb" ) )
train = cPickle.load( open( "train_v1.pkl", "rb" ) )
test = cPickle.load( open( "test_v1.pkl", "rb" ) )
passive = cPickle.load( open( "pasive_v1.pkl", "rb" ) )

In [5]:
def normalizer(data):
    for dim in range(data.shape[0]):
        for row in range(1,data.shape[1]):
            for ele in range(1,data.shape[2]):
                if abs(data[dim,row,ele] - 0) < 1e-10:
                    data[dim,row,ele] = 0.0
                elif abs(data[dim,row,ele] - 1) < 1e-10:
                    data[dim,row,ele] = 1.0
                elif abs(data[dim,row,ele] - 2) < 1e-10 or abs(data[dim,row,ele] - 3) < 1e-10 or abs(data[dim,row,ele] - 4) < 1e-10:
                    data[dim,row,ele] = 2.0
                elif abs(data[dim,row,ele] - 5) < 1e-10 or abs(data[dim,row,ele] - 6) < 1e-10 or abs(data[dim,row,ele] - 7) < 1e-10:
                    data[dim,row,ele] = 3.0
                elif abs(data[dim,row,ele] - 8) < 1e-10 or abs(data[dim,row,ele] - 9) < 1e-10 or abs(data[dim,row,ele] - 10) < 1e-10:
                    data[dim,row,ele] = 4.0
                elif abs(data[dim,row,ele] - 11) < 1e-10 or abs(data[dim,row,ele] - 12) < 1e-10 or abs(data[dim,row,ele] - 13) < 1e-10:
                    data[dim,row,ele] = 5.0

In [10]:
normalizer(train)
normalizer(test)
normalizer(passive)

NameError: name 'normalizer' is not defined

In [13]:
reputation = []
for t_idx in range(1,passive.shape[1]):
    temp,temp_idx = 0,0
    for p_idx in range(1,passive.shape[1]):
        if t_idx is p_idx:
            continue
        nonzero_arow, nonzero_prow = passive[-1,t_idx].nonzero()[0], passive[-1,p_idx].nonzero()[0]
        common_idx = np.intersect1d(nonzero_arow, nonzero_prow)
        reliable,Ra = 0,0
        if common_idx[1:].shape[0] > 0: 
            mx,my = passive[-1,1,common_idx[1:]].mean(),passive[-1,1,common_idx[1:]].mean()
            rae,r_dash = [],[]
            for ele in common_idx[1:]:
                py = (mx - my) + passive[-1,p_idx,ele]
                diff = abs(py - passive[-1,t_idx,ele])

                if diff >= 0 and diff <= 0.5 :
                    rae.append(2.0)
                elif diff > 0.5 and diff <= 1.0:
                    rae.append(1.0)
                elif diff > 1.0 and diff <= 1.5:
                    rae.append(0.0)
                elif diff > 1.5 and diff <= 2.0:
                    rae.append(-1.0)
                elif diff > 2.0 :
                    rae.append(-2.0)

                p_dash = (mx - my) + passive[-1,t_idx,ele]
                diff = abs(p_dash - passive[-1,p_idx,ele])
                if diff > 1.0 and diff <= 1.5:
                    r_dash.append(0.0)

            rae,S_dash_zero = array(rae),len(r_dash)
            Sap,San,S_zero = rae[rae > 0], rae[rae < 0], rae[abs(rae - 0) < 1e-10].shape[0]
            Nap,Nan = Sap.shape[0], San.shape[0]
            
            Ra = 1/rae.shape[0] * (Nap - Nan)/(Nap + Nan + 2)
            if rae.shape[0] > 0:
                reliable = 1 - ((S_zero + S_dash_zero)/(rae.shape[0] + (rae.shape[0])))
        else:
            reliable,Ra = 1, 0
        temp += reliable * Ra
        temp_idx += 1
    reputation.append(temp/temp_idx)

In [14]:
len(reputation)

433

In [15]:
def matr(shp):
    temp = zeros(shp)
    temp[0,:] = range(shp[1])
    temp[:,0] = range(shp[0])
    return temp

def simlarity(row1,passive_user,k):
    k_neighbr,idx,arr = np.zeros(passive_user.shape[0]),0,passive_user[:,0]

    for row2 in passive_user[:,1:]:
        temp1,temp2,temp3 = 0,0,0
        mu1 = np.sum([row1], axis=1)[0]/row1.nonzero()[0].shape[0]
        mu2 = np.sum([row2], axis=1)[0]/row2.nonzero()[0].shape[0]
        for i in range(len(row1)):
            if abs(row1[i] - 0) > 1e-10 and abs(row2[i] - 0) > 1e-10:
                temp1 += ((row1[i] - mu1) * (row2[i] - mu2))
                temp2 += (row1[i] - mu1) * (row1[i] - mu1)
                temp3 += (row2[i] - mu2) * (row2[i] - mu2)
        
        k_neighbr[idx] = 0 if abs(temp2 - 0) < 1e-10 or abs(temp3 - 0) < 1e-10 else temp1/(sqrt(temp2) * sqrt(temp3))
        idx+=1
    
    return arr[k_neighbr.argsort()][-k:][::-1]

def k_neighbor(data,passive_user,k=50):
    users,idx = matr((data.shape[0],k+1)),1
    for user in data[1:,1:]:
        users[idx,1:],users[idx,0] = simlarity(user,passive_user[1:],k),data[idx,0]
        idx += 1    
    return users

In [16]:
relation_matrix = k_neighbor(train[-1],passive[-1],80)

In [17]:
def diffmapping(diff,rea,sat,unsat):

    if diff >= 0 and diff <= 0.5 :
        rae.append(2.0)
        sat.append(1)
        unsat.append(0)
    elif diff > 0.5 and diff <= 1.0:
        rae.append(1.0)
        sat.append((1.0 + 2.0)/4)
        unsat.append((1 - (3.0/4)))
    elif diff > 1.0 and diff <= 1.5:
        rae.append(0.0)
        sat.append((0.0 + 2.0)/4)
        unsat.append((1 - (2.0/4)))
    elif diff > 1.5 and diff <= 2.0:
        rae.append(-1.0)
        sat.append((-1.0 + 2.0)/4)
        unsat.append((1 - (1.0/4)))
    elif diff > 2.0 :
        rae.append(-2.0)
        sat.append(0)
        unsat.append(1)
        
    return rae,sat,unsat

def satisfy(sat,sat2,unsat,unsat2):
    
    sns,sus,unu,uuu,snu,suu,uns,uus = 0,0,0,0,0,0,0,0
    for s1,s2,u1,u2 in zip(sat,sat2,unsat,unsat2):

        sns += min(s1,s2)
        sus += max(s1,s2)
        unu += min(u1,u2)
        uuu += max(u1,u2)
        snu += min(s1,u2)
        suu += max(s1,u2)
        uns += min(u1,s2)
        uus += max(u1,s2)   
        
    SS = 0 if abs(sus-0) < 1e-10 else sns/sus    
    UU = 0 if abs(uuu-0) < 1e-10 else unu/uuu
    SU = 0 if abs(suu-0) < 1e-10 else snu/suu
    US = 0 if abs(uus-0) < 1e-10 else uns/uus 
    
    agr,disagr = (SS + UU)/2, (SU + US)/2
    rec = (1 - disagr)*agr
    
    return rec

In [18]:
trust = matr(relation_matrix.shape)
trust[:,0] = relation_matrix[:,0]

passive_dict = {}
for idx,value in enumerate(passive[-1,:,0]):
    passive_dict[value] = idx

Flag = 0
for t_idx in range(1,train.shape[1]):
    recip = []
    for r_idx in range(1,relation_matrix.shape[1]):
        if t_idx is r_idx:
            continue
        p_idx = passive_dict[relation_matrix[t_idx,r_idx]]
        nonzero_arow, nonzero_prow = train[-1,t_idx].nonzero()[0], passive[-1,p_idx].nonzero()[0]
        common_idx = np.intersect1d(nonzero_arow, nonzero_prow)
        reliable,rec = 0,0
        if common_idx[1:].shape[0] > 0: 

            mx,my = train[-1,1,common_idx[1:]].mean(),passive[-1,1,common_idx[1:]].mean()
            rae,r_dash,sat,unsat,sat2,unsat2 = [],[],[],[],[],[]
            for ele in common_idx[1:]:
                py = (mx - my) + passive[-1,p_idx,ele]
                diff = abs(py - train[-1,t_idx,ele])
                rae,sat,unsat = diffmapping(diff,rae,sat,unsat)
                p_dash = (mx - my) + train[-1,t_idx,ele]
                diff = abs(p_dash - passive[-1,p_idx,ele])
                r_dash,sat2,unsat2 = diffmapping(diff,r_dash,sat2,unsat2)
            
            rec = satisfy(sat,sat2,unsat,unsat2)
            
            rae,S_dash_zero = array(rae), len(r_dash) 
            S_zero = rae[abs(rae - 0) < 1e-10].shape[0]
            if rae.shape[0] > 0:
                reliable = 1 - ((S_zero + S_dash_zero)/(rae.shape[0] + (rae.shape[0])))
        else:
            reliable = 1
        
        trust[t_idx,r_idx] = reliable * rec

# From 80 nearest neighbor choosing top 50 via trust

In [19]:
trust_50,relation_matrix_50 = matr((trust.shape[0],51)),matr((relation_matrix.shape[0],51))
trust_50[:,0],relation_matrix_50[:,0] = trust[:,0],relation_matrix[:,0]
for idx in range(1,relation_matrix.shape[0]):
    row_relation,row_trust,alpha = relation_matrix[idx,1:],trust[idx,1:],trust[idx,1:].argsort()
    relation_matrix_50[idx,1:],trust_50[idx,1:] = row_relation[alpha][-50:][::-1],row_trust[alpha][-50:][::-1]

# 3rd phase

In [20]:
movie_dict = {}
for idx,value in enumerate(passive[-1,0,:]):
    movie_dict[value] = idx

In [21]:
def passive_details(user_id,movie_id,movie_dict = movie_dict,passive_dataset = passive):
    return passive_dataset[:,user_id,movie_dict[movie_id]]

def value_details(T_idx,P_idx,reputation,trust_50,relation_matrix_50,passive_dict):
    if P_idx in relation_matrix_50[T_idx,1:]:
        idx = np.where(relation_matrix_50[T_idx] == P_idx)[0][0]
        return trust_50[T_idx,idx], True
    else:
        return reputation[P_idx - 1], False

In [22]:
def Fr_funtion(train,passive,trust_50,relation_matrix_50,passive_dict):
    Train_user_list = []
    Train_y_list = []
    for T_idx in range(1,train.shape[1]):
    #     Tracer()()
        rated_movies_idx = train[-1,T_idx,1:].nonzero()[0] + 1
        if abs(len(rated_movies_idx) - 0) < 1e-10:
                continue
        Fr_list,Fx_list = [],[]
        for movie_idx in rated_movies_idx:
            Tr,Rep,Tr_sum,Rep_sum = array([0.0,0,0,0]),array([0.0,0,0,0]),array([0.0,0,0,0]),array([0.0,0,0,0])
            movie_train_id = train[-1,0,movie_idx]
            for P_idx in range(1,passive.shape[1]):
                trusted_rating = passive_details(P_idx,movie_train_id)[:-1]
                value,trt = value_details(T_idx,P_idx,reputation,trust_50,relation_matrix_50,passive_dict)
                if 0 in trusted_rating:
                    continue
                if trt is True:
                    Tr += value*trusted_rating
                    Tr_sum += value
                else:
                    Rep += value*trusted_rating
                    Rep_sum += value

            if 0 in Tr_sum and 0 in Rep_sum: 
                pass
            elif not(0 in Tr_sum) and not(0 in Rep_sum):
                Fr_list.append((0.6*Tr/Tr_sum) + (0.4*(Rep)/Rep_sum))     
                Fx_list.append(train[-1,T_idx,movie_idx])
            elif not(0 in Tr_sum) and 0 in Rep_sum:    
                Fr_list.append((0.6*Tr/Tr_sum))
                Fx_list.append(train[-1,T_idx,movie_idx])
            elif 0 in Tr_sum and not(0 in Rep_sum):
                Fr_list.append((0.6*Rep/Rep_sum))
                Fx_list.append(train[-1,T_idx,movie_idx])
        Train_user_list.append(array(Fr_list))
        Train_y_list.append(array(Fx_list))
    return Train_user_list,Train_y_list

In [23]:
semi_train,semi_y_train = Fr_funtion(train,passive,trust_50,relation_matrix_50,passive_dict)

In [24]:
semi_test,semi_y_test = Fr_funtion(test,passive,trust_50,relation_matrix_50,passive_dict)

In [25]:
# cPickle.dump( semi_train, open( "semi_train.pkl", "wb" ) )
# cPickle.dump( semi_y_train, open( "semi_y_train.pkl", "wb" ) )
# cPickle.dump( semi_test, open( "semi_test.pkl", "wb" ) )
# cPickle.dump( semi_y_test, open( "semi_y_test.pkl", "wb" ) )

semi_train = cPickle.load( open( "semi_train.pkl", "rb" ) )
semi_y_train = cPickle.load( open( "semi_y_train.pkl", "rb" ) )
semi_test = cPickle.load( open( "semi_test.pkl", "rb" ) )
semi_y_test = cPickle.load( open( "semi_y_test.pkl", "rb" ) )

# Train_x,Test_x

In [26]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from IPython.core.debugger import Tracer

Using TensorFlow backend.


In [28]:
model = Sequential()
model.add(Dense(3, activation='relu', input_dim=4))
model.add(Dense(1,activation='linear'))
model.compile(optimizer='rmsprop',loss='mse')

In [29]:
model.save_weights('model.h5')

In [45]:
def train_error(model,train_x,train,test_x,test):
#     Tracer()()
    lst = []
    list_err = []
    for i in range(len(train_x)):
        try:
            Tr_X = array(train_x[i])
            Tr_Y = array(train[i])
            Tt_X = array(test_x[i])
            Tt_Y = array(test[i])
            model.load_weights('model.h5')
            model.fit(Tr_X, Tr_Y, epochs=150, batch_size=int(Tr_X.shape[0]*0.7), verbose=0)

            err,tp,fp,fn,tn = 0.0,0.0,0.0,0.0,0.0
        
            predicted = model.predict(Tt_X, batch_size=Tt_X.shape[0])
            for Pred,Actual in zip(predicted,Tt_Y):
#                 print(Pred,Actual)
                if Pred >= 3 and Actual >= 3:
                    tp+=1.0
                elif Pred < 3 and Actual >= 3:
                    fn+=1.0
                elif Pred >= 3 and Actual < 3:
                    fp+=1.0
                else:
                    tn+=1.0
            
            prec = tp/(tp+fp)
            recal = tp/(tp+fn)
            f_meas = (2*recal*prec)/(recal + prec)
            err = np.absolute((predicted - Tt_Y)).mean(axis=0)[0]
            list_err.append([err,prec,recal,f_meas])
        
        except:
            print("error : ",i)
                    
    list_err = array(list_err)
    
    return np.mean(list_err,axis=0)

In [46]:
a = train_error(model,semi_train,semi_y_train,semi_test,semi_y_test)

error :  34


In [47]:
a

array([ 1.18482629,  0.93107382,  0.83312258,  0.86750932])

In [ ]:
# lst_err = []
# list_file = cPickle.load( open( "list_file.pkl", "rb" ) )
# for i in range(len(list_file)):
#     print("HI",i)
#     lst_err.append(train_error(model,semi_train,semi_y_train,semi_test,semi_y_test))

In [52]:
type(array(semi_test[1]))

numpy.ndarray

In [45]:
len(semi_y_train[1])

16

In [59]:
data = np.random.random((1000, 100))


In [62]:
data.shape

(1000, 100)

In [64]:
semi_train[0].shape

(35, 4)